In [1]:
import numpy as np 
import pandas as pd 
import requests
import json
from functools import reduce

In [2]:
df = pd.read_csv('dialect_dataset.csv') 
df.head()

,id,dialect
0,1175358310087892992,IQ
1,1175416117793349632,IQ
2,1175450108898565888,IQ
3,1175471073770573824,IQ
4,1175496913145217024,IQ


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  int64 
 1   dialect  458197 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


**In first we test how to post to API by id number** 


In [4]:
url = "https://recruitment.aimtechnologies.co/ai-tasks"
r = requests.post(url,json=['1055620304465215616','1057418989293485952'])


In [5]:
r.text

'{"1055620304465215616":"@MahmoudWaked7 @maganenoo في طريق مطروح مركز بهيج  والمركز الي الي جمبه اسمه ايه😂😂","1057418989293485952":"@mycousinvinnyys @hanyamikhail1 متهيالي دي شكولاته الهالوين  فين المحل ده"}'

In [6]:
r.json()

{'1055620304465215616': '@MahmoudWaked7 @maganenoo في طريق مطروح مركز بهيج  والمركز الي الي جمبه اسمه ايه😂😂',
 '1057418989293485952': '@mycousinvinnyys @hanyamikhail1 متهيالي دي شكولاته الهالوين  فين المحل ده'}

then know what value counts of  every dialect 

In [7]:
df.dialect.value_counts()

EG    57636
PL    43742
KW    42109
LY    36499
QA    31069
JO    27921
LB    27617
SA    26832
AE    26296
BH    26292
OM    19116
SY    16242
DZ    16183
IQ    15497
SD    14434
MA    11539
YE     9927
TN     9246
Name: dialect, dtype: int64

In [8]:
df['id'] = df['id'].astype(str) # convert the id number to string 

In [9]:
countries_name = list(set(df.dialect.values)) # get the name of each counrty in list 

**create two funcations (get_counrty and get text ) 
- get counrty to get each coun

In [10]:
def get_country(country_name) :
    
    """" this funcation to get data frame of country with id and dialect 
    has one paramter it is the country name as string
    """"
    country = df.loc[lambda df: df['dialect'] == country_name, :]
    country = country.reset_index(drop=True)
    return country


In [11]:
df_EG = get_country('EG')
df_EG

,id,dialect
0,1130199209695744000,EG
1,1137520472420233216,EG
2,1137717993172033536,EG
3,1137810705514684416,EG
4,1125394999997284352,EG
...,...,...
57631,1057418989293485952,EG
57632,1057775213113495552,EG
57633,1055620304465215616,EG
57634,1056641788851744768,EG


In [12]:
def get_text (country) :
    country_text=[]
    for i in range(0,len(country),100):
        list_of_string = country.id[i:100+i]
        r = requests.post(url,json=[i for i in list_of_string])
        text=r.json()
        df =  pd.DataFrame(list(text.items()),columns=['id','text'])
        #print(c)
        country_text.append(df)
    #country_text = reduce(lambda  left,right: pd.merge(left,right,on=['text'],how='outer'),country_text)
    country_text = pd.concat(country_text)
    country_text = country_text.reset_index(drop=True)
    country_text = country_text.merge(country,on=['id'],how='right')
    return country_text
    

In [13]:
new_data = []
for i in countries_name :
    country = get_country(i)
    country = get_text(country)
    new_data.append(country)
    
new_data = pd.concat(new_data)
new_data = new_data.reset_index(drop=True)    
    
    

In [14]:
new_data

,id,text,dialect
0,1121525449874604032,#الخميس_الونيس\n\nانتو ملاحظين أن الغلطان هالأ...,KW
1,1123251810955865984,من الحين قاعد اضحك على اللي راح يصور ويكتب آخر...,KW
2,1124892962859683840,نرجو عدم التأليف بارسال التهاني برمضان او العي...,KW
3,1127803659478290432,@araaaatv ماراح تتعداك حوبة هالمسكين مهما بلغت...,KW
4,1129889371375755264,انت سد الفراغ اللي بشنبك بعدين سد فراغها العاطفي,KW
...,...,...,...
458192,1057418989293485952,@mycousinvinnyys @hanyamikhail1 متهيالي دي شكو...,EG
458193,1057775213113495552,@mycousinvinnyys @diamond_mamy ماعرفش لايصح غ...,EG
458194,1055620304465215616,@MahmoudWaked7 @maganenoo في طريق مطروح مركز ب...,EG
458195,1056641788851744768,@FayedSasa للاسف شعب يجري وراء الازمات بيحبوا ...,EG


In [15]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458197 non-null  object
 1   text     458197 non-null  object
 2   dialect  458197 non-null  object
dtypes: object(3)
memory usage: 10.5+ MB


In [17]:
new_data.dialect.value_counts()

EG    57636
PL    43742
KW    42109
LY    36499
QA    31069
JO    27921
LB    27617
SA    26832
AE    26296
BH    26292
OM    19116
SY    16242
DZ    16183
IQ    15497
SD    14434
MA    11539
YE     9927
TN     9246
Name: dialect, dtype: int64

In [19]:
new_data.to_csv('new_dialect_dataset.csv')